Disney Dataset Creation (w/ Python BeautifulSoup)

In [4]:
#import necessary libraries
import requests
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd
import json


Scrape & clean a list of disney wikipedia pages to create a dataset to further analyze

Task #1 : Get info box (store in Python dictionary)

In [5]:
def get_movie_data(url) : 
    sleep(0.1)
    response = requests.get(url)
    # Convert to a beautiful soup object
    soup = BeautifulSoup(response.content, 'lxml')

    # Print out the HTML
    try:
        info_rows = soup.find(class_="infobox vevent").find_all("tr")
        def get_content_value(row_data):
                if row_data.find("li"):
                    return [li.get_text(" ", strip=True).replace("\xa0", " ") for li in row_data.find_all("li")]
                else:
                    return row_data.get_text(" ", strip=True).replace("\xa0", " ")
        movie_info = {}
        for index, row in enumerate(info_rows):
            if index == 0:
                movie_info['title'] = row.find("th").get_text(" ", strip=True)
            elif index == 1:
                continue
            else:
                content_key = row.find("th").get_text(" ", strip=True)
                content_value = get_content_value(row.find("td"))
                movie_info[content_key] = content_value
            
        return movie_info
    except:
        return None

get_movie_data('https://en.wikipedia.org/wiki/Toy_Story_3')['title']

'Toy Story 3'

Task #2 : get info box for all movies

In [6]:
res = requests.get('https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films')
soup = BeautifulSoup(res.content, 'lxml')

results = soup.select('.wikitable.sortable i a')
urls = ['https://en.wikipedia.org/' + url['href'] for url in results]

movies_data = [get_movie_data(url) for url in urls if get_movie_data(url) != None]

df = pd.DataFrame(movies_data)
df.to_json('./Output/disney_dataset.json', orient='index')